---
title: "Data Analysis"
subtitle: "Comprehensive Data Cleaning & Exploratory Analysis of Job Market Trends"
author:
  - name: Caroline O'Sullivan
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
  - name: Bhargavi Manyala
    affiliations:
      - ref: bu
  - name: Taylor Luckenbill
    affiliations:
      - ref: bu      
bibliography: bibliography.bib
csl: csl/econometrica.csl
format: 
  html:
    toc: true
    number-sections: true
    df-print: paged
engine: jupyter
jupyter: python3
execute:
  echo: true      
  eval: true      
  output: true    
  freeze: auto 
---


# Import Data
Importing dataset using Pyspark..The code shows the schema and few rows of the dataset.


In [ ]:
from pyspark.sql import SparkSession

# Start a Spark session
spark = SparkSession.builder.appName("JobPostingsAnalysis").getOrCreate()

    
# Load the CSV file into a Spark DataFrame
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiLine", "true") \
    .option("escape", "\"") \
    .csv("lightcast_job_postings.csv")

df.createOrReplaceTempView("jobs")


# Show schema
df.printSchema()

In [ ]:
#| echo : false
df.show(5, truncate=False)


# Data Cleaning & Preprocessing

## Drop Unnecessary Columns

### Redundant Columns

* Drop: NAICS2, NAICS2_NAME, NAICS3, NAICS3_NAME, NAICS4, NAICS4_NAME, NAICS5, NAICS5_NAME, NAICS6, NAICS6_NAME
* Retain: NAICS_2022_2, NAICS_2022_2_NAME through NAICS_2022_6, NAICS_2022_6_NAME
* Drop: SOC_2, SOC_2_NAME, SOC_3, SOC_3_NAME, SOC_4, SOC_4_NAME, SOC_5, SOC_5_NAME
* Retain: SOC_2021_2, SOC_2021_2_NAME, SOC_2021_3, SOC_2021_3_NAME, SOC_2021_4, SOC_2021_4_NAME, SOC_2021_5, SOC_2021_5_NAME
* Drop: CITY, COUNTY, STATE, REMOTE_TYPE, EMPLOYMENT_TYPE (numeric codes when text names exist)
* Drop : COUNTY_OUTGOING, COUNTY_NAME_OUTGOING, COUNTY_INCOMING, COUNTY_NAME_INCOMING, MSA_OUTGOING, MSA_NAME_OUTGOING, MSA_INCOMING, MSA_NAME_INCOMING (geographic mobility data)
* Retain: LOCATION, CITY_NAME, COUNTY_NAME, STATE_NAME, MSA, MSA_NAME, REMOTE_TYPE_NAME, EMPLOYMENT_TYPE_NAME (text-based fields)
* Drop: TITLE_RAW, TITLE, TITLE_NAME, COMPANY_RAW (unprocessed versions)
* Retain: TITLE_CLEAN, COMPANY_NAME (cleaned/processed versions)
* Drop: EDUCATION_LEVELS, MIN_EDULEVELS, MAX_EDULEVELS (numeric codes)
* Retain : EDUCATION_LEVELS_NAME, MIN_EDULEVELS_NAME, MAX_EDULEVELS_NAME (descriptive names)
* Drop: LOT_V6_SPECIALIZED_OCCUPATION, LOT_V6_SPECIALIZED_OCCUPATION_NAME, LOT_V6_OCCUPATION, LOT_V6_OCCUPATION_NAME,  LOT_V6_OCCUPATION_GROUP, LOT_V6_OCCUPATION_GROUP_NAME, LOT_V6_CAREER_AREA, LOT_V6_CAREER_AREA_NAME (version-specific)
* Retain: LOT_CAREER_AREA, LOT_CAREER_AREA_NAME, LOT_OCCUPATION, LOT_OCCUPATION_NAME, LOT_SPECIALIZED_OCCUPATION, LOT_SPECIALIZED_OCCUPATION_NAME, LOT_OCCUPATION_GROUP, LOT_OCCUPATION_GROUP_NAME (current standard)

### Irrelevant Columns

* Drop: LAST_UPDATED_DATE, LAST_UPDATED_TIMESTAMP, DUPLICATES, SOURCE_TYPES, SOURCES, ACTIVE_URLS, ACTIVE_SOURCES_INFO (system tracking only)
* Retain: Core job posting fields only
* Drop: ID, URL, BODY (technical identifier, link, and full text not needed for analysis)
* Retain: Main job details (title, company, salary, location)
* Drop: CIP6, CIP6_NAME, CIP4, CIP4_NAME, CIP2, CIP2_NAME (education codes)
  ONET, ONET_NAME, ONET_2019, ONET_2019_NAME (extra occupation codes)
  LIGHTCAST_SECTORS, LIGHTCAST_SECTORS_NAME, (alternative grouping)
* Retain: Standard NAICS, SOC, and LOT classifications only
* Drop: SKILLS, SKILLS_NAME, SPECIALIZED_SKILLS, SPECIALIZED_SKILLS_NAME, CERTIFICATIONS, CERTIFICATIONS_NAME, COMMON_SKILLS,      COMMON_SKILLS_NAME, SOFTWARE_SKILLS, SOFTWARE_SKILLS_NAME (too detailed text fields)
* Retain: Key job attributes only
* Drop: MODELED_EXPIRED, MODELED_DURATION (estimates, not real values)
* Retain: POSTED, EXPIRED, DURATION (actual posting data)

### Why are we removing multiple versions of NAICS/SOC codes?

The dataset includes both older and newer versions of industry (NAICS) and occupation (SOC) codes. Keeping multiple versions creates duplication and confusion because the same job could be categorized under two systems. To avoid this, we retain only the most recent standards — NAICS 2022 and SOC 2021 — which reflect the current labor market.

### How will this improve analysis?

* Improves analysis as follows:
* Ensures clarity by eliminating overlapping or outdated categories.
* Maintains consistency so all jobs follow the same classification.
* Improves efficiency by reducing extra columns that don’t add value.
* Increases accuracy since results are based on the most up-to-date classification systems.

### Dropping Unnecessary Columns(Python Implementation)

In [ ]:
columns_to_drop = [
    "NAICS2", "NAICS2_NAME", "NAICS3", "NAICS3_NAME",
    "NAICS4", "NAICS4_NAME", "NAICS5", "NAICS5_NAME",
    "NAICS6", "NAICS6_NAME","ID","URL","BODY",
    "SOC_2", "SOC_2_NAME", "SOC_3", "SOC_3_NAME",
    "SOC_4", "SOC_4_NAME", "SOC_5", "SOC_5_NAME",
    "CITY", "COUNTY", "STATE", "REMOTE_TYPE", "EMPLOYMENT_TYPE",
    "COUNTY_OUTGOING", "COUNTY_NAME_OUTGOING",
    "COUNTY_INCOMING", "COUNTY_NAME_INCOMING",
    "MSA_OUTGOING", "MSA_NAME_OUTGOING",
    "MSA_INCOMING", "MSA_NAME_INCOMING",
    "TITLE_RAW", "TITLE", "TITLE_NAME", "COMPANY_RAW",
    "EDUCATION_LEVELS", "MIN_EDULEVELS", "MAX_EDULEVELS",
    "LOT_V6_SPECIALIZED_OCCUPATION", "LOT_V6_SPECIALIZED_OCCUPATION_NAME",
    "LOT_V6_OCCUPATION", "LOT_V6_OCCUPATION_NAME",
    "LOT_V6_OCCUPATION_GROUP", "LOT_V6_OCCUPATION_GROUP_NAME",
    "LOT_V6_CAREER_AREA", "LOT_V6_CAREER_AREA_NAME",
    "LAST_UPDATED_DATE", "LAST_UPDATED_TIMESTAMP",
    "DUPLICATES", "SOURCE_TYPES", "SOURCES",
    "ACTIVE_URLS", "ACTIVE_SOURCES_INFO",
    "CIP2", "CIP2_NAME", "CIP4", "CIP4_NAME", "CIP6", "CIP6_NAME",
    "ONET", "ONET_NAME", "ONET_2019", "ONET_2019_NAME",
    "LIGHTCAST_SECTORS", "LIGHTCAST_SECTORS_NAME",
    "SKILLS", "SKILLS_NAME", "SPECIALIZED_SKILLS", "SPECIALIZED_SKILLS_NAME",
    "CERTIFICATIONS", "CERTIFICATIONS_NAME",
    "COMMON_SKILLS", "COMMON_SKILLS_NAME",
    "SOFTWARE_SKILLS", "SOFTWARE_SKILLS_NAME",
    "MODELED_EXPIRED", "MODELED_DURATION"
]

df = df.drop(*columns_to_drop)
df.show(truncate=False)
df.count()


## Handling Missing Values(Numerical Python Implementation)

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

numeric_cols = [
    "DURATION", "COMPANY", "MIN_YEARS_EXPERIENCE", "MAX_YEARS_EXPERIENCE",
    "SALARY", "SALARY_TO", "SALARY_FROM", "MSA",
    "LOT_CAREER_AREA", "LOT_OCCUPATION", "LOT_SPECIALIZED_OCCUPATION",
    "LOT_OCCUPATION_GROUP", "NAICS_2022_2", "NAICS_2022_3",
    "NAICS_2022_4", "NAICS_2022_5", "NAICS_2022_6"
]

pandas_num = df.select(numeric_cols).toPandas()

msno.heatmap(pandas_num)
plt.title("Missing Values Heatmap - Numeric")
plt.show()

pandas_num[numeric_cols] = pandas_num[numeric_cols].fillna(
    pandas_num[numeric_cols].median(numeric_only=True)
)
pandas_num.head()

## Handling Missing Values(Categorical Python Implementation)

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

categorical_cols = [
    "COMPANY_NAME", "EDUCATION_LEVELS_NAME",
    "MIN_EDULEVELS_NAME", "MAX_EDULEVELS_NAME", "EMPLOYMENT_TYPE_NAME",
    "REMOTE_TYPE_NAME", "ORIGINAL_PAY_PERIOD", "LOCATION", "CITY_NAME",
    "COUNTY_NAME", "MSA_NAME", "STATE_NAME", "TITLE_CLEAN",
    "SOC_2021_2", "SOC_2021_2_NAME", "SOC_2021_3", "SOC_2021_3_NAME",
    "SOC_2021_4", "SOC_2021_4_NAME", "SOC_2021_5", "SOC_2021_5_NAME",
    "LOT_CAREER_AREA_NAME", "LOT_OCCUPATION_NAME",
    "LOT_SPECIALIZED_OCCUPATION_NAME", "LOT_OCCUPATION_GROUP_NAME",
    "NAICS_2022_2_NAME", "NAICS_2022_3_NAME", "NAICS_2022_4_NAME",
    "NAICS_2022_5_NAME", "NAICS_2022_6_NAME"
]

pandas_cat = df.select(categorical_cols).toPandas()

msno.heatmap(pandas_cat)
plt.title("Missing Values Heatmap - Categorical")
plt.show()

pandas_cat[categorical_cols] = pandas_cat[categorical_cols].fillna("Unknown")

pandas_cat.head()

## Handling Missing Values(Date Python Implementation)

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt
import pandas as pd

date_cols = ["POSTED", "EXPIRED"]
pandas_date = df.select(date_cols).toPandas()

msno.heatmap(pandas_date)
plt.title("Missing Values Heatmap - Date Columns")
plt.show()

for col in date_cols:
    pandas_date[col] = pd.to_datetime(pandas_date[col], errors="coerce")
    pandas_date[col] = pandas_date[col].fillna(pandas_date[col].mode()[0])

pandas_date.head()

## Handling Missing Values(Combined Dataframe Python Implementation)

In [ ]:
import pandas as pd

pandas_handled_missing_values = pd.concat([pandas_num, pandas_cat, pandas_date], axis=1)

pandas_handled_missing_values.head()

##  Duplicates Removal(Python Implementation)

In [ ]:
pandas_cleaned = pandas_handled_missing_values.drop_duplicates(
    subset=["TITLE_CLEAN", "COMPANY_NAME", "LOCATION","POSTED"],
    keep="first"
)

pandas_cleaned
